# Amortizirana časovna zahtevnost

Operacija v podatkovni strukturi ima _amortizirano časovno zahtevnost_ $A(n)$, če vsako zaporedje $n$ takih operacij porabi največ $A(n) ⋅ n$ enot časa.

## Primeri

### Dvojiški števec

Števec predstavimo s tabelo $B$, ki predstavlja število $\sum_{i = 0}^\infty B[i] \cdot 2^i$.

In [ ]:
class Counter:
    def __init__(self, velikost):
        self.bits = [0] * velikost

    def flip(self, i):
        self.bits[i] = 1 - self.bits[i]

    def __str__(self):
        return "".join(str(bit) for bit in reversed(self.bits))

    def __int__(self):
        return sum(bit * (2**i) for i, bit in enumerate(self.bits))

    def __getitem__(self, i):
        return self.bits[i]

In [24]:
VELIKOST = 4
KORAKI = 10

stevec = Counter(VELIKOST)
for korak in range(KORAKI):
    print(f"{stevec} = {int(stevec)}")
    i = 0
    while stevec[i] == 1:
        stevec.flip(i)
        i += 1
    stevec.flip(i)
print(f"{stevec} = {int(stevec)}")

0000 = 0
0001 = 1
0010 = 2
0011 = 3
0100 = 4
0101 = 5
0110 = 6
0111 = 7
1000 = 8
1001 = 9
1010 = 10


In [ ]:
import plotly.graph_objects as go
from collections import defaultdict


class CostTracker:
    def __init__(self, title):
        self.title = title
        self.costs = defaultdict(int)

    def add_cost(self, step, cost):
        self.costs[step] += cost

    def plot(self):
        steps = sorted(self.costs.keys())
        costs = [self.costs[s] for s in steps]

        if len(steps) <= 128:
            fig = go.Figure(data=go.Bar(x=steps, y=costs))
        else:
            fig = go.Figure(data=go.Scatter(x=steps, y=costs, mode="lines"))
        fig.update_layout(xaxis_title="Korak", yaxis_title=self.title).show()

In [29]:
VELIKOST = 8
KORAKI = 100

spremembe_bitov = CostTracker("Spremenjeni biti")
counter = Counter(VELIKOST)

for korak in range(KORAKI):
    i = 0
    while counter[i] == 1:
        counter.flip(i)
        spremembe_bitov.add_cost(step=korak, cost=1)
        i += 1
    counter.flip(i)
    spremembe_bitov.add_cost(step=korak, cost=1)

spremembe_bitov.plot()

Vsak `increment` porabi $O(\log n)$ časa. Torej $n$ operacij porabi $O(n \log n)$ časa. Znamo bolje?

### Dinamična tabela

#### Lastna implementacija

In [ ]:
class DynamicTable:
    def __init__(self, cost_tracker):
        self._size = 0
        self._data = []
        self._cost_tracker = cost_tracker

    @property
    def _capacity(self):
        return len(self._data)

    def __len__(self):
        return self._size

    def __getitem__(self, index):
        if index < 0:
            index += self._size
        return self._data[index]

    def append(self, value):
        if self._size == self._capacity:
            self._resize(max(1, 2 * self._capacity))
        self._data[self._size] = value
        self._size += 1
        self._cost_tracker.add_cost(step=self._size, cost=1)

    def _resize(self, new_capacity):
        new_data = [None] * new_capacity
        for i in range(self._size):
            new_data[i] = self._data[i]
        self._cost_tracker.add_cost(step=self._size, cost=self._size)
        self._data = new_data

In [ ]:
VELIKOST_TABELE = 10_000

stevilo_vstavljenih = CostTracker("Vstavljeni elementi")
tabela = DynamicTable(stevilo_vstavljenih)

for korak in range(1, VELIKOST_TABELE + 1):
    tabela.append(korak)

stevilo_vstavljenih.plot()

Vsak `append` porabi $O(n)$ časa. Torej $n$ operacij porabi $O(n^2)$ časa. Znamo bolje?

#### Vgrajena implementacija

In [ ]:
import sys
import time

STEVILO_POSKUSOV = 1
VELIKOST_TABELE = 1_000

porabljen_cas = CostTracker("Vstavljeni elementi")

for _ in range(STEVILO_POSKUSOV):
    tabela = []
    for korak in range(VELIKOST_TABELE):
        kapaciteta = sys.getsizeof(tabela)
        zacetek = time.time()
        tabela.append(korak)
        konec = time.time()
        nova_kapaciteta = sys.getsizeof(tabela)
        if nova_kapaciteta > kapaciteta:
            print(
                f"{korak}: {kapaciteta} → {nova_kapaciteta} - {nova_kapaciteta / kapaciteta:.3f}×"
            )
        porabljen_cas.add_cost(step=korak, cost=konec - zacetek)

porabljen_cas.plot()

## Agregacijska metoda

Naj bo $\mathcal{S}$ množica stanj naše podatkovne strukture in $op_i : \mathcal{S} → \mathcal{S}$ zaporedje operacij, ki spreminjajo podatkovno strukturo kot:

$$
S_0 \stackrel{op_1}{→} S_1 \stackrel{op_2}{→} S_2 \stackrel{op_3}{→} ⋯ \stackrel{op_n}{→} S_n
$$

Naj bo $C_i$ čas, ki ga zahteva operacija $op_i$.

Pri agregacijski metodi amortizirano časovno zahtevnost izračunamo tako, da seštejemo čase vseh operacij $\sum_{i = 1}^n C_i$ in vsoto ocenimo navzgor na $A(n) ⋅ n$ za nek $A(n)$. Potem po definiciji velja $\sum_{i = 1}^n C_i ≤ A(n) ⋅ n$, zato je $A(n)$ amortizirana časovna zahtevnost.

### Dvojiški števec

V $n$ operacijah `increment` se:
- bit $B[0]$ spremeni na vsakem koraku, torej $n$-krat;
- bit $B[1]$ spremeni na vsakem drugem koraku, torej $⌊n / 2⌋$-krat;
- bit $B[2]$ spremeni na vsakem četrtem koraku, torej $⌊n / 4⌋$-krat;
- ⋮
- bit $B[i]$ spremeni na vsakem $2^i$-koraku, torej $⌊n / 2^{\log n}⌋$-krat

Torej je skupno število sprememb v $n$ korakih:
$$
    n + \left⌊\frac{n}{2}\right⌋ + \left⌊\frac{n}{4}\right⌋ + ⋯ + \left⌊\frac{n}{2^{\log n}}\right⌋
    =
    \sum_{i = 0}^∞ \left⌊\frac{n}{2^i}\right⌋
    ≤
    \sum_{i = 0}^∞ \frac{n}{2^i}
    =
    2 n
    
$$

Ocenili smo $A(\_) = 2$, torej je amortizirana časovna zahtevnost $O(1)$.

### Dinamična tabela

V $n$ operacijah `append` se:
- na vsakem koraku vstavi en element, torej $n$ vstavljanj;
- na koraku $2^i$ vstavimo še dodatnih $2^i$ elementov v novo tabelo.

Torej je skupno število vstavljanj v $n$ korakih:
$$
    n + 2^1 + 2^2 + 2^3 + ⋯ + 2^{⌊\log n⌋}
    =
    n + \sum_{i = 1}^{⌊\log n⌋} 2^i
    =
    n + 2^{⌊\log n⌋ + 1} - 1
    ≤
    n + 2^{\log n + 1}
    =
    3 n
$$

Amortizirana časovna zahtevnost je tudi tu $O(1)$.

## Računovodska metoda

Vsaka operacija lahko prevzame del stroška prihodnjih operacij. To storimo tako, da med izvajanjem beležimo porabo (virtualno, ne v dejanski implementaciji), operacije pa potem bodisi „plačajo davek“ za prihodnje operacije oziroma so plačana iz poprej vplačanih davkov.

Formalno gledano vsaki operaciji $op_i$ v zaporedju:

$$
S_0 \stackrel{op_1}{→} S_1 \stackrel{op_2}{→} S_2 \stackrel{op_3}{→} ⋯ \stackrel{op_n}{→} S_n
$$

priredimo amortizirano zahtevnost $A_i$ tako, da:
1. $A_i$ ocenimo lažje kot $C_i$,
2. velja $∀ j ≤ n. \sum_{i = 1}^j A_i ≥ \sum_{i = 1}^j C_i$, torej da smo na vsaki točki poskrbeli, da bo dejanski strošek poplačan iz amortizirane zahtevnosti prejšnjih operacij.

### Dvojiški števec

Vsak bit dobi svojo „žepnino“, cene operacij pa so sledeče:

- Sprememba bita $0 → 1$ plača dve enoti, eno porabi za spremembo, drugo pa si shrani za naslednjo morebitno spremembo $1 → 0$.
- Sprememba bita $1 → 0$ ne plača ničesar, saj je bila sprememba že pokrita v spremembi $0 → 1$.

Na vsakem koraku lahko naredimo več sprememb $1 → 0$, ampak _samo eno_ spremembo $0 → 1$, zato na vsakem koraku velja $A_i = 2$, kar nas zopet privede do iste amortizirane časovne zahtevnosti.

In [30]:
EURO = "€"
NI_EURA = " "

VELIKOST = 4
KORAKI = 10

stevec = Counter(VELIKOST)
zepnina = [NI_EURA] * VELIKOST
for korak in range(KORAKI):
    print("".join(zepnina))
    print(f"{stevec} = {int(stevec)}")
    i = 0
    while stevec[i] == 1:
        stevec.flip(i)
        assert zepnina[i] == EURO
        zepnina[i] = NI_EURA
        i += 1
    stevec.flip(i)
    zepnina[i] = EURO
print("".join(zepnina))
print(f"{stevec} = {int(stevec)}")

    
0000 = 0
€   
0001 = 1
 €  
0010 = 2
€€  
0011 = 3
  € 
0100 = 4
€ € 
0101 = 5
 €€ 
0110 = 6
€€€ 
0111 = 7
   €
1000 = 8
€  €
1001 = 9
 € €
1010 = 10


Recimo, da števec želimo tudi zmanjševati. V tem primeru lahko z ustreznim zaporedjem operacij prehajamo med stanjema $100⋯0$ in $011⋯1$, zato je tudi amortizirana časovna zahtevnost $O(\log n)$. Lahko pa smo zviti in števec predstavimo s števcema $P$ in $N$, ki predstavljata število $P - N$ (tako kot $ℤ$ predstavimo z ekvivalenčnimi razredi parov $ℕ × ℕ$).

Števec povečamo tako, da kot prej $P$ povečamo za $1$, le da pri (zadnji) spremembi $0 → 1$ pogledamo, če je na ustreznem mestu v $N$ že $1$. Če je, števca pokrajšamo in tam popravimo $1 → 0$.

In [ ]:
def povecaj(stevec, nasprotni):
    i = 0
    while stevec[i] == 1:
        stevec.flip(i)
        i += 1
    if nasprotni[i] == 1:
        nasprotni.flip(i)
    else:
        stevec.flip(i)


VELIKOST = 5
KORAKI = "+++--+"

pozitivni = Counter(VELIKOST)
negativni = Counter(VELIKOST)

# Nastavimo P = 10001 in N = 01100 kot v Ericksonu 9.3
for _ in range(17):
    povecaj(pozitivni, pozitivni)
for _ in range(12):
    povecaj(negativni, negativni)

for korak in KORAKI:
    print(f"{pozitivni} - {negativni} = {int(pozitivni)} - {int(negativni)} = {int(pozitivni) - int(negativni)}")
    if korak == "+":
        povecaj(pozitivni, negativni)
    else:
        povecaj(negativni, pozitivni)
print(f"{pozitivni} - {negativni} = {int(pozitivni)} - {int(negativni)} = {int(pozitivni) - int(negativni)}")


10001 - 01100 = 17 - 12 = 5
10010 - 01100 = 18 - 12 = 6
10011 - 01100 = 19 - 12 = 7
10000 - 01000 = 16 - 8 = 8
10000 - 01001 = 16 - 9 = 7
10000 - 01010 = 16 - 10 = 6
10001 - 01010 = 17 - 10 = 7


V tem primeru agregatna metoda odpove, saj ne moremo več predvideti, kakšno bo videti zaporedje korakov. Pri računovodski metodi pa podobno kot prej pridemo do amortizirane časovne zahtevnosti 2.


### Dinamična tabela

Vsako vstavljanje stane tri enote:

- Ena enota se porabi za vstavljanje elementa v tabelo.
- Druga enota se porabi za prvo kasnejše prestavljanje v večjo tabelo.
- Tretja enota se porabi za prestavljanje nekega že prestavljenega elementa v večjo tabelo.

### Disjunktne množice

In [ ]:
class DisjointSet:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [0] * size

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]

    def union(self, x, y):
        px = self.find(x)
        py = self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            self.parent[px] = py
        elif self.rank[px] > self.rank[py]:
            self.parent[py] = px
        else:
            self.parent[py] = px
            self.rank[px] += 1
        return True

Dokažimo, da $m$ klicev metode `find` na $n$ elementih porabi $O(m \log^* n) + O(n \log^* n)$ časa. Pri tem je običajno $n ≤ m$, torej je amortizirana časovna zahtevnost $O(\log^* n)$.

Naj bo $I_i = \{ x ∈ \mathbb{N} | \log^* x = i \}$:

$$
I_0 = [1, 1] \qquad
I_1 = [2, 2] \qquad
I_2 = [3, 4] \qquad
I_3 = [5, 16] \qquad
I_4 = [17, 2^{16}] \qquad
I_5 = [2^{16} + 1, 2^{2^{16}}]
$$

Ko vozlišče ranga $r ∈ [ℓ + 1, 2^ℓ]$ preneha biti koren, mu damo $2^ℓ$ žepnine. Vsak koren ranga $k$ ima vsaj $2^k$ potomcev. Zato mora biti vozlišč ranga $k$ kvečjemu $n / 2^k$, torej jih je na tem intervalu kvečjemu

$$
\frac{n}{2^{ℓ + 1}} + \frac{n}{2^{ℓ + 2}} + ⋯ + \frac{n}{2^{2^ℓ}} < \frac{n}{2^ℓ}
$$

zato za njih porabimo največ $n$ žepnine. Ker je intervalov $\log^* n$, smo podelili $O(n \log^* n)$ žepnine.

Vsaka operacija `find` porabi toliko časa, kot je dolga njena pot $x = y_0 → y_1 → ⋯ → y_r$. Vrednosti $\mathop{\mathrm{rang}(y_i)}$ in $\mathop{\mathrm{rang}(y_{i + 1})}$ sta lahko:
- na različnih intervalih (kar se lahko zgodi največ $(\log^* n)$-krat)
- na istem intervalu $[ℓ + 1, 2^ℓ]$, pri čemer bomo strošek plačali iz žepnine $y_i$. Ker bomo $y_i$ prevezali največ $2^ℓ$-krat, ima dovolj žepnine.

## Potencialna metoda

Potencialna metoda po navdihu fizike sledi celotni „žepnini“ v potencialni energiji podatkovne strukture. Ob hitrih operacijah potencial akumuliramo, ob počasnih pa ga porabljamo. Formalno definiramo _potencial_ $Φ : \mathcal{S} → ℝ^+$. Za zaporedje operacij

$$
S_0 \stackrel{op_1}{→} S_1 \stackrel{op_2}{→} S_2 \stackrel{op_3}{→} ⋯ \stackrel{op_n}{→} S_n
$$

pišemo $Φ_i = Φ(S_i)$ ter definiramo amortizirano časovno zahtevnost kot $A_i = C_i + Φ_i - Φ_{i - 1}$. Tedaj velja:

$$\sum_{i = 1}^n C_i = \sum_{i = 1}^n (A_i - Φ(S_i) + Φ(S_{i - 1})) = (\sum_{i = 1}^n A_i) - Φ_n + Φ_0$$

Če definiramo $\Phi_0 = 0$ (razmislite, zakaj to lahko vedno storimo), iz $Φ_n ≥ 0$ sledi $\sum_{i = 1}^n C_i ≤ \sum_{i = 1}^n A_i$.

V primerjavi z računovodsko metodo je potencialna metoda bolj eksplicitna, saj je potencial odvisen samo od trenutnega stanja podatkovne strukture in ne od zaporedja operacij, ki so pripeljale do nje (do istega stanja lahko pridemo tudi z različnimi zaporedji operacij in posledično z različnimi žepninami). Poleg tega je potencialna metoda lokalna, saj se iz istega razloga pri izračunu amortizirane časovne zahtevnosti posamezne operacije lahko osredotočimo samo na stanje pred in po operaciji, ne glede na to, kaj se je dogajalo prej.

### Dvojiški števec

Pri števcu lahko potencial definiramo kot število bitov, ki so enaki $1$:

$$Φ(B) = \#\{ i \mid B[i] = 1 \}$$

Vemo, da je $C_i = \#(0 → 1) + \#(1 → 0)$, kjer z $\#(1 → 0)$ označimo število sprememb $0 → 1$. Po drugi strani je $\Phi_i - \Phi_{i - 1} = \#(0 → 1) - \#(1 → 0)$, saj za spremembe $0 → 1$ potencial povečujemo, za spremembe $1 → 0$ pa jih zmanjšujemo. Tedaj enostavno velja

$$A_i = (\#(0 → 1) + \#(1 → 0)) + (\#(0 → 1) - \#(1 → 0)) = 2 \#(0 → 1) = 2$$

saj v vsakem koraku naredimo samo eno spremembo $0 → 1$.


In [39]:
VELIKOST = 4
KORAKI = 10

stevec = Counter(VELIKOST)
potencial = 0

for korak in range(KORAKI):
    print(f"{stevec} = {int(stevec)}")
    cena = 0
    i = 0
    while stevec[i] == 1:
        stevec.flip(i)
        cena += 1
        i += 1
    stevec.flip(i)
    cena += 1
    novi_potencial = sum(stevec)
    print(
        f"A"
        f" = (c = {cena}) + (novi = {novi_potencial}) - (stari = {potencial})"
        f" = {cena + novi_potencial - potencial}"
    )
    potencial = novi_potencial
print(f"{stevec} = {int(stevec)}")

0000 = 0
A = (c = 1) + (novi = 1) - (stari = 0) = 2
0001 = 1
A = (c = 2) + (novi = 1) - (stari = 1) = 2
0010 = 2
A = (c = 1) + (novi = 2) - (stari = 1) = 2
0011 = 3
A = (c = 3) + (novi = 1) - (stari = 2) = 2
0100 = 4
A = (c = 1) + (novi = 2) - (stari = 1) = 2
0101 = 5
A = (c = 2) + (novi = 2) - (stari = 2) = 2
0110 = 6
A = (c = 1) + (novi = 3) - (stari = 2) = 2
0111 = 7
A = (c = 4) + (novi = 1) - (stari = 3) = 2
1000 = 8
A = (c = 1) + (novi = 2) - (stari = 1) = 2
1001 = 9
A = (c = 2) + (novi = 2) - (stari = 2) = 2
1010 = 10


Če lahko števec tudi zmanjšujemo prek para števcev $P, N$, pa lahko potencial definiramo kot:

$$Φ(P, N) = \#\{ i \mid P[i] = 1 \} + \#\{ i \mid N[i] = 1 \}$$

Tako kot prej velja

$$A_i = (\#(0 → 1) + \#(1 → 0)) + (\#(0 → 1) - \#(1 → 0)) = 2 \#(0 → 1)$$

le da tu štejemo vse spremembe v obeh števcih. Še vedno pa velja, da v vsakem koraku naredimo samo eno spremembo $0 → 1$, zato je $A_i = 2$.

### Dinamična tabela

Če s $K(T)$ označimo kapaciteto tabele $T$, z $N(T)$ pa število njenih elementov, lahko potencial definiramo v odvisnosti od števila elementov, ki že segajo čez polovico kapacitete:

$$Φ(T) = 2 (N(T) - K(T) / 2) = 2 N(T) - K(T)$$

Ker je tabela vedno vsaj na pol polna, je $Φ(T) ≥ 0$. Enakost $Φ(T) = 0$ velja, kadar je $K(T) = 2 N(T)$, torej ravno po razširitvi kapacitete.

Če operacija $op_i$ ne poveča kapacitete ter označimo $N_i = N(T_i)$ in $K_i = N(T_i)$, velja

$$
\begin{align*}
A_i
    &= C_i + Φ_i - Φ_{i - 1} \\
    &= 1 + (2 N_i - K_i) - (2 N_{i - 1} - K_{i - 1}) \\
    &= 1 + 2 (N_i - N_{i - 1}) - (K_i - K_{i - 1}) \\
    &= 1 + 2 ⋅ 1 - 0 \\
    &= 3
\end{align*}
$$

Če pa operacija poveča kapaciteto, pa imamo $N_{i - 1} = K_{i - 1}$ in $K_i = 2 K_{i - 1}$, zato velja

$$
\begin{align*}
A_i
    &= C_i + Φ_i - Φ_{i - 1} \\
    &= N_i + 2 (N_i - N_{i - 1}) - (K_i - K_{i - 1}) \\
    &= N_i + 2 ⋅ 1 - K_{i - 1} \\
    &= (1 + K_{i - 1}) + 2 ⋅ 1 - K_{i - 1} \\
    &= 3
\end{align*}
$$

Tudi tu lahko potencialno metodo posplošimo na situacije, ki jih z agregatno in računovodsko metodo ne moremo zajeti. Recimo, da želimo tabelo tudi zmanjševati. Najprej moramo paziti, da tabele ne zmanjšamo že takrat, ko število elementov pade pod polovico kapacitete, saj bi sicer z izmenjujočim dodajanjem in odstranjevanjem neprestano povečevali in zmanjševali tabelo. Namesto tega se odločimo, da tabelo zmanjšamo, kadar število elementov pade pod četrtino kapacitete.

Naj bo $α(T) = N(T)/K(T)$ delež zasedenosti. Definirajmo

$$Φ(T) = \begin{cases}
    2 N(T) - K(T); &α(T) ≥ 1/2 \\
    K(T)/2 - N(T); &α(T) < 1/2
\end{cases}$$

Hitro lahko preverimo, da je potencial prazne tabele enak $0$ in da je potencial nenegativen. Za vajo lahko izračunamo, da je pri dodajanju amortizirana časovna zahtevnost dodajanja pri $α(T) < 1/2$ enaka 0, v ostalih primerih pa 3 kot prej. Pri odstranjevanju je amortizirana zahtevnost brez zmanjševanja tabele enaka 2, ob zmanjševanju pa 1. Več v CLRS 17.4.2.

## Semidinamična podatkovna struktura